In [1]:
import codecs
from math import sqrt

In [3]:
class recommender:
    
    def __init__(self, data, k=1, metric='pearson', n=5):
        self.k = k
        self.n = n
        self.username2id = {}
        self.userid2name = {}
        self.productid2name = {} 
        self.frequencies = {}
        self.deviations = {}
        self.metric = metric
        if self.metric == 'pearson':
            self.fn = self.pearson
      
        if type(data).__name__ == 'dict':
            self.data = data

    def convertProductID2name(self, id):
      if id in self.productid2name:
         return self.productid2name[id]
      else:
         return id


    def userRatings(self, id, n):
      print ("Ratings for " + self.userid2name[id])
      ratings = self.data[id]
      print(len(ratings))
      ratings = list(ratings.items())[:n]
      ratings = [(self.convertProductID2name(k), v)
                 for (k, v) in ratings]
      ratings.sort(key=lambda artistTuple: artistTuple[1],
                   reverse = True)      
      for rating in ratings:
         print("%s\t%i" % (rating[0], rating[1]))


    def showUserTopItems(self, user, n):
      items = list(self.data[user].items())
      items.sort(key=lambda itemTuple: itemTuple[1], reverse=True)
      for i in range(n):
         print("%s\t%i" % (self.convertProductID2name(items[i][0]),
                           items[i][1]))
            
    def loadMovieLens(self, path=''):
      self.data = {}
      i = 0
      f = codecs.open(path + "u.data", 'r', 'ascii')
      for line in f:
         i += 1
         fields = line.split('\t')
         user = fields[0]
         movie = fields[1]
         rating = int(fields[2].strip().strip('"'))
         if user in self.data:
            currentRatings = self.data[user]
         else:
            currentRatings = {}
         currentRatings[movie] = rating
         self.data[user] = currentRatings
      f.close()
      f = codecs.open(path + "u.item", 'r', 'iso8859-1', 'ignore')
      
      for line in f:
         i += 1
         fields = line.split('|')
         mid = fields[0].strip()
         title = fields[1].strip()
         self.productid2name[mid] = title
      f.close()
      
      f = open(path + "u.user")
      for line in f:
         i += 1
         fields = line.split('|')
         userid = fields[0].strip('"')
         self.userid2name[userid] = line
         self.username2id[line] = userid
      f.close()
      print(i)

                    
        
    def computeDeviations(self):
      for ratings in self.data.values():
         for (item, rating) in ratings.items():
            self.frequencies.setdefault(item, {})
            self.deviations.setdefault(item, {})                    
            for (item2, rating2) in ratings.items():
               if item != item2:
                  self.frequencies[item].setdefault(item2, 0)
                  self.deviations[item].setdefault(item2, 0.0)
                  self.frequencies[item][item2] += 1
                  self.deviations[item][item2] += rating - rating2
        
      for (item, ratings) in self.deviations.items():
         for item2 in ratings:
            ratings[item2] /= self.frequencies[item][item2]


    def slopeOneRecommendations(self, userRatings):
      recommendations = {}
      frequencies = {}
      for (userItem, userRating) in userRatings.items():
         for (diffItem, diffRatings) in self.deviations.items():
            if diffItem not in userRatings and \
               userItem in self.deviations[diffItem]:
               freq = self.frequencies[diffItem][userItem]
               recommendations.setdefault(diffItem, 0.0)
               frequencies.setdefault(diffItem, 0)
               recommendations[diffItem] += (diffRatings[userItem] +
                                             userRating) * freq
               
               frequencies[diffItem] += freq
      recommendations =  [(self.convertProductID2name(k),
                           v / frequencies[k])
                          for (k, v) in recommendations.items()]
      
      recommendations.sort(key=lambda artistTuple: artistTuple[1],
                           reverse = True)
      
      return recommendations[:50]
        
    def pearson(self, rating1, rating2):
      sum_xy = 0
      sum_x = 0
      sum_y = 0
      sum_x2 = 0
      sum_y2 = 0
      n = 0
      for key in rating1:
         if key in rating2:
            n += 1
            x = rating1[key]
            y = rating2[key]
            sum_xy += x * y
            sum_x += x
            sum_y += y
            sum_x2 += pow(x, 2)
            sum_y2 += pow(y, 2)
      if n == 0:
         return 0
      
      denominator = sqrt(sum_x2 - pow(sum_x, 2) / n) * \
                    sqrt(sum_y2 - pow(sum_y, 2) / n)
      if denominator == 0:
         return 0
      else:
         return (sum_xy - (sum_x * sum_y) / n) / denominator


    def computeNearestNeighbor(self, username):
      distances = []
      for instance in self.data:
         if instance != username:
            distance = self.fn(self.data[username],
                               self.data[instance])
            distances.append((instance, distance))
      
      distances.sort(key=lambda artistTuple: artistTuple[1],
                     reverse=True)
      return distances

    def recommend(self, user):
      
      recommendations = {}
      
      nearest = self.computeNearestNeighbor(user)
      
      userRatings = self.data[user]
      
      totalDistance = 0.0
      for i in range(self.k):
         totalDistance += nearest[i][1]
      
      for i in range(self.k):
          
         weight = nearest[i][1] / totalDistance
         
         name = nearest[i][0]
         
         neighborRatings = self.data[name]
         
         for artist in neighborRatings:
            if not artist in userRatings:
               if artist not in recommendations:
                  recommendations[artist] = neighborRatings[artist] * \
                                            weight
               else:
                  recommendations[artist] = recommendations[artist] + \
                                            neighborRatings[artist] * \
                                            weight
      
      recommendations = list(recommendations.items())[:self.n]
      recommendations = [(self.convertProductID2name(k), v)
                         for (k, v) in recommendations]
      
      recommendations.sort(key=lambda artistTuple: artistTuple[1],
                           reverse = True)
      return recommendations

In [4]:
r = recommender(0)

In [5]:
r.loadMovieLens('/home/ubuntu/my_data/ml-100k/')

102625


In [6]:
r.showUserTopItems('1',50)

When Harry Met Sally... (1989)	5
Jean de Florette (1986)	5
Godfather, The (1972)	5
Big Night (1996)	5
Manon of the Spring (Manon des sources) (1986)	5
Sling Blade (1996)	5
Breaking the Waves (1996)	5
Terminator 2: Judgment Day (1991)	5
Searching for Bobby Fischer (1993)	5
Maya Lin: A Strong Clear Vision (1994)	5
Mighty Aphrodite (1995)	5
Bound (1996)	5
Full Monty, The (1997)	5
Chasing Amy (1997)	5
Ridicule (1996)	5
Nightmare Before Christmas, The (1993)	5
Three Colors: Red (1994)	5
Professional, The (1994)	5
Priest (1994)	5
Welcome to the Dollhouse (1995)	5
Terminator, The (1984)	5
Graduate, The (1967)	5
Dead Poets Society (1989)	5
Amadeus (1984)	5
Henry V (1989)	5
Star Wars (1977)	5
Gattaca (1997)	5
Wallace & Gromit: The Best of Aardman Animation (1996)	5
Groundhog Day (1993)	5
Truth About Cats & Dogs, The (1996)	5
Horseman on the Roof, The (Hussard sur le toit, Le) (1995)	5
Jurassic Park (1993)	5
Return of the Jedi (1983)	5
Hudsucker Proxy, The (1994)	5
Remains of the Day, The (1993)

In [7]:
r.computeDeviations()

In [8]:
r.slopeOneRecommendations(r.data['1'])

[(u'Entertaining Angels: The Dorothy Day Story (1996)', 6.375),
 (u'Aiqing wansui (1994)', 5.849056603773585),
 (u'Boys, Les (1997)', 5.644970414201183),
 (u"Someone Else's America (1995)", 5.391304347826087),
 (u'Santa with Muscles (1996)', 5.380952380952381),
 (u'Great Day in Harlem, A (1994)', 5.275862068965517),
 (u'Little City (1998)', 5.236363636363636),
 (u'Pather Panchali (1955)', 5.209612817089453),
 (u'Saint of Fort Washington, The (1993)', 5.201754385964913),
 (u"Some Mother's Son (1996)", 5.154471544715447),
 (u'Faust (1994)', 5.127450980392157),
 (u'Angel Baby (1995)', 5.021671826625387),
 (u'Butcher Boy, The (1998)', 4.966101694915254),
 (u'Spanish Prisoner, The (1997)', 4.966101694915254),
 (u'Butcher Boy, The (1998)', 4.966101694915254),
 (u'Brothers in Trouble (1995)', 4.966101694915254),
 (u'Anna (1996)', 4.932),
 (u'Bitter Sugar (Azucar Amargo) (1996)', 4.897689768976898),
 (u'They Made Me a Criminal (1939)', 4.853932584269663),
 (u'Mina Tannenbaum (1994)', 4.8479809

In [9]:
r.slopeOneRecommendations(r.data['25'])

[(u'Aiqing wansui (1994)', 5.674418604651163),
 (u'Boys, Les (1997)', 5.523076923076923),
 (u'Star Kid (1997)', 5.5),
 (u'Santa with Muscles (1996)', 5.260869565217392),
 (u'Faust (1994)', 5.204081632653061),
 (u'Great Day in Harlem, A (1994)', 5.2),
 (u'Angel Baby (1995)', 5.184466019417476),
 (u'Saint of Fort Washington, The (1993)', 5.178571428571429),
 (u'Celestial Clockwork (1994)', 5.159090909090909),
 (u'Anna (1996)', 5.139240506329114),
 (u"Some Mother's Son (1996)", 5.064102564102564),
 (u'Nightwatch (1997)', 5.0),
 (u'Little City (1998)', 5.0),
 (u'They Made Me a Criminal (1939)', 4.956521739130435),
 (u'Pather Panchali (1955)', 4.950331125827814),
 (u'Mina Tannenbaum (1994)', 4.9393939393939394),
 (u'Butcher Boy, The (1998)', 4.925925925925926),
 (u'Spanish Prisoner, The (1997)', 4.925925925925926),
 (u'Butcher Boy, The (1998)', 4.925925925925926),
 (u'Brothers in Trouble (1995)', 4.925925925925926),
 (u"Someone Else's America (1995)", 4.837209302325581),
 (u'Entertaining An

In [10]:
r.userRatings('1',10)

Ratings for 1|24|M|technician|85711

272
When Harry Met Sally... (1989)	5
Jean de Florette (1986)	5
Godfather, The (1972)	5
Pink Floyd - The Wall (1982)	4
Unbearable Lightness of Being, The (1988)	4
Indiana Jones and the Last Crusade (1989)	4
Bram Stoker's Dracula (1992)	3
Field of Dreams (1989)	3
M*A*S*H (1970)	3
Room with a View, A (1986)	2
